In [0]:
from google.colab import drive
drive.mount("/content/drive",force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import argparse
import os
import numpy as np
import math
from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler 
from torchvision import datasets
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch

In [0]:
if not os.path.exists('./2nd_question'):
    os.mkdir('./2nd_question')

In [0]:
def Display(i):
    i = 0.5 * (i + 1)
    i = i.clamp(0, 1)
    i = i.view(i.size(0), 3, 28, 28)
    return i

In [0]:
def getParameters():
  epochs = 300
  batch_size = 128
  learn_rate = 0.001  
  return epochs,batch_size,learn_rate

In [0]:
epochs,batch_size,learn_rate = getParameters()

In [0]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encodedOutput = None
        self.encoder = nn.Sequential(
            nn.Linear(2352, 128),
            nn.ReLU(True),
            nn.Linear(128, 100))
        self.decoder = nn.Sequential(
            nn.Linear(100, 128),
            nn.ReLU(True), nn.Linear(128, 2352), nn.Tanh())

    def forward(self, i):
        i = self.encoder(i)
        self.encodedOutput = i.detach().numpy()
        i = self.decoder(i)
        return i

In [0]:
model = autoencoder()

In [0]:
criterion = nn.MSELoss()

In [0]:
optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate, weight_decay=0.0001)

In [0]:
root_dir = 'drive/My Drive/faces94'
files = glob(f"{root_dir}/**/**/*.jpg")

In [0]:
vam = {"female": 0,"male":1,"malestaff": 2}

In [0]:
transform_loader = transforms.Compose([
    transforms.RandomResizedCrop((28, 28)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5],[0.5])
])

In [0]:
All_Images = torch.stack([transform_loader(Image.open(i)) for i in files])
All_Labels = torch.tensor([vam[i.split('/')[-3]] for i in files])

In [0]:
def getDataloaders(All_Images,All_Labels):
  train_data, test_data = train_test_split(range(len(All_Images)), test_size=0.2, random_state=102)
  train_images = All_Images[train_data]
  train_label = All_Labels[train_data]
  test_images = All_Images[test_data]
  test_label = All_Labels[test_data]
  train_data = TensorDataset(train_images, train_label)
  test_data = TensorDataset(test_images, test_label)
  sampleTrain = RandomSampler(train_data)
  sampleTest = SequentialSampler(test_data)
  trainDataLoader = DataLoader(train_data, sampler=sampleTrain, batch_size=1)
  testDataLoader = DataLoader(test_data, sampler=sampleTest, batch_size=1)
  
  return trainDataLoader,testDataLoader

In [0]:
trainDataLoader,testDataLoader=getDataloaders(All_Images,All_Labels)

In [0]:
def run_model(epochs,trainDataLoader,testDataLoader,optimizer,model):
  for epoch in range(epochs):
      for data in trainDataLoader:
          image, _ = data
          image = image.view(image.size(0), -1)
          image = Variable(image)
          output = model(image)
          loss = criterion(output, image)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
      print(
                "{ Epoch --> %d of %d}  { Loss --> %.4f }"
                % (epoch+1, epochs,loss.item())
            )
      if epoch % 10 == 0:
          pic = Display(output.data)
          save_image(pic, './2nd_question/image_{}.png'.format(epoch))


In [0]:
run_model(epochs,trainDataLoader,testDataLoader,optimizer,model)

{ Epoch --> 1 of 2}  { Loss --> 0.1045 }
{ Epoch --> 2 of 2}  { Loss --> 0.0354 }


In [0]:
model.eval()
def getCsvFile(epochs,trainDataLoader,testDataLoader,optimizer,model):
  for data in testDataLoader:
      image, _ = data
      image = image.view(image.size(0), -1)
      image = Variable(image)
      output = model(image)
      print(model.encodedOutput)
      print(model.encodedOutput.shape)
      np.savetxt("output1.csv", model.encodedOutput, delimiter=",")
      break

In [0]:
getCsvFile(epochs,trainDataLoader,testDataLoader,optimizer,model)

[[-6.93641186e-01  1.70134115e+00  4.95633554e+00 -4.02711064e-01
   4.25515294e-01 -1.30522001e+00 -8.36690366e-01  2.29952514e-01
  -1.28079093e+00  4.56474675e-03  1.32276809e+00 -1.21076083e+00
   2.62694669e+00  1.06806219e+00 -1.07411551e+00  4.35308695e-01
   1.28463912e+00  3.75299072e+00  1.34669936e+00 -1.50669205e+00
   4.28432405e-01  1.87507796e+00 -2.78146482e+00 -5.87898970e-01
   3.90572691e+00  5.19764721e-01  1.46613669e+00 -4.33666766e-01
  -5.98424435e-01 -1.07216251e+00  3.06974977e-01 -2.68230200e-01
   1.53375959e+00  1.17892241e+00  2.13186443e-03 -9.71496344e-01
   2.94820309e+00  4.20448571e-01  3.36573839e+00 -8.11377645e-01
  -1.19745135e-01 -4.71483380e-01 -8.49280417e-01  2.79488039e+00
   1.03305392e-01 -1.70848119e+00  1.07696664e+00  1.79874241e+00
   4.87156820e+00 -9.09039140e-01  2.60342574e+00  4.85552728e-01
  -1.83913380e-01 -1.91383314e+00  1.49460578e+00 -2.17123866e+00
  -3.22212905e-01 -1.06152105e+00  5.55241287e-01 -6.09887064e-01
  -2.85512